# Assemble the Features

We want to assemble our data in to a data frame of features; for now I'm going to try to make something including:

* Position player performance data (~3 numbers)
* Position player position
* Team salary data
* Team performance for position (previous year)
* Team value lost for position (from previous year, using FAs)

We'll try doing it in stages

In [161]:
# Bring in packages and connect to database
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
from scipy.stats import zscore

# Set postgres username/password, and connection specifics
username = 'postgres'
password = 'S@ndw1ches'     # change this
host     = 'localhost'
port     = '5432'            # default port that postgres listens on
db_name  = 'mlb_fa_db'

engine = create_engine( 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, db_name) )

In [162]:
# Make a quick querying function
def pullFullTable(table, engine):
    '''Quick little function for pulling a full table'''
    
    query = 'select * from {}'.format(table)
    
    # Execute the query with context manager
    with engine.connect() as con:
        results = con.execute(query)
        fetched_data = pd.DataFrame(results.fetchall())
        fetched_data.columns = results.keys()
        
    return fetched_data

## Task 1: Grab Batting data and filter it by only free agents

We'll do it in 5 stages:

1. Pull batting data and shorten its columns to just the ones I want
2. Pull the "people" data to get the first/last names for batting data
3. Join batting and people to get all the data JUST for our desired years
3. Pull the "free_agents" data
5. Join "batting" and new free_agents/people to filter batting by only free agents

In [163]:
print(engine.table_names())

['free_agents', 'pitching', 'batting', 'salary', 'people', 'position_team_war', 'appearances', 'teams', 'pitcher_team_war', 'payrolls']


In [164]:
# Create our query
batting_data = pullFullTable('batting', engine)
    
batting_data.head()

,index,playerID,yearID,stint,teamID,lgID,G,AB,R,H,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,76643,abbotje01,1998,1,CHA,AL,89,244,33,68,...,41.0,3.0,3.0,9,28.0,1.0,0.0,2.0,5.0,2.0
1,76644,abbotji01,1998,1,CHA,AL,5,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,76645,abbotku01,1998,1,OAK,AL,35,123,17,33,...,9.0,2.0,1.0,10,34.0,0.0,1.0,1.0,1.0,3.0
3,76646,abbotku01,1998,2,COL,NL,42,71,9,18,...,15.0,0.0,0.0,2,19.0,0.0,1.0,0.0,2.0,2.0
4,76647,abbotpa01,1998,1,SEA,AL,4,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [165]:
# Drop non-numeric team/league columns
batting_data.drop(['teamID','lgID'], axis = 1)

# Add data from players who had multiple stints and 
batting_totals = batting_data.groupby(['playerID','yearID'], as_index= False).sum()

In [166]:
# Create new variables and select only them (fill NaN with 0)
batting_totals['OBP'] = (batting_totals['H'] + 
                       batting_totals['BB'] + 
                       batting_totals['HBP']).divide(batting_totals['AB'] + 
                                                   batting_totals['BB'] + 
                                                   batting_totals['HBP'] + 
                                                   batting_totals['SF']).fillna(0)

batting_totals['SLG'] = (batting_totals['H'] + 
                         batting_totals['2B'] + 
                         2 * batting_totals['3B'] + 
                         3 * batting_totals['HR']).divide(batting_totals['AB']).fillna(0)

batting_trimmed = batting_totals[['playerID', 'yearID', 'G', 'OBP', 'SLG', 'HR', 'RBI']]

# Fill "NaN" values for OBP/SLG with 0
print(batting_trimmed.shape)
print(batting_trimmed.head())
print(batting_trimmed[batting_trimmed['playerID'] == 'pujolal01'])

(25365, 7)
    playerID  yearID   G  OBP  SLG  HR  RBI
0  aardsda01    2004  11  0.0  0.0   0  0.0
1  aardsda01    2006  45  0.0  0.0   0  0.0
2  aardsda01    2007  25  0.0  0.0   0  0.0
3  aardsda01    2008  47  0.0  0.0   0  0.0
4  aardsda01    2009  73  0.0  0.0   0  0.0
        playerID  yearID    G       OBP       SLG  HR    RBI
18417  pujolal01    2001  161  0.402963  0.610169  37  130.0
18418  pujolal01    2002  157  0.394074  0.561017  34  127.0
18419  pujolal01    2003  157  0.439416  0.666667  43  124.0
18420  pujolal01    2004  154  0.414740  0.657095  46  123.0
18421  pujolal01    2005  161  0.430000  0.609137  41  117.0
18422  pujolal01    2006  143  0.430599  0.671028  49  137.0
18423  pujolal01    2007  158  0.428571  0.568142  32  103.0
18424  pujolal01    2008  148  0.461778  0.652672  37  116.0
18425  pujolal01    2009  160  0.442857  0.658451  47  135.0
18426  pujolal01    2010  159  0.414286  0.596252  42  118.0
18427  pujolal01    2011  147  0.365591  0.540587  37 

In [167]:
# Standardize the numerical columns by year
numerical = ['G', 'OBP', 'SLG', 'HR', 'RBI']

batting_trimmed[numerical] = batting_trimmed.groupby('yearID')[numerical].transform(zscore)

# Take a look at the output
print(batting_trimmed.shape)
print(batting_trimmed.head())
print(batting_trimmed[batting_trimmed['playerID'] == 'pujolal01'])

(25365, 7)
    playerID  yearID         G       OBP       SLG        HR       RBI
0  aardsda01    2004 -0.926802 -1.128818 -1.021286 -0.520478 -0.613383
1  aardsda01    2006 -0.223818 -1.096651 -1.018416 -0.510545 -0.606120
2  aardsda01    2007 -0.623055 -1.074030 -0.948662 -0.505253 -0.600185
3  aardsda01    2008 -0.146466 -1.085320 -0.912276 -0.496440 -0.599853
4  aardsda01    2009  0.388188 -1.070321 -1.006028 -0.506939 -0.601370
        playerID  yearID         G       OBP       SLG        HR       RBI
18417  pujolal01    2001  2.193543  1.141557  1.615230  3.637957  3.709145
18418  pujolal01    2002  2.089353  1.156760  1.559438  3.652412  3.852653
18419  pujolal01    2003  2.102354  1.271069  1.857574  4.707794  3.641002
18420  pujolal01    2004  2.069087  1.336778  1.943523  4.956622  3.615371
18421  pujolal01    2005  2.189724  1.362913  1.529887  4.658596  3.554547
18422  pujolal01    2006  1.802831  1.322061  1.937764  5.258251  3.979439
18423  pujolal01    2007  2.149787  1.

/home/matt/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


### Now we'll pull the People and Free Agents and join People to Batting, and 

Note: I tried to do the join directly with SQL and it got mad, so I'm going to do it here instead

In [168]:
# Bring in people and free agents
people = pullFullTable('people', engine)
free_agents = pullFullTable('free_agents', engine)

print(people.shape, free_agents.shape)
print(people.columns, free_agents.columns)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 25 elements

In [ ]:
# Join people to batting, adding ONLY nameLast/nameFirst
people_trimmed = people[['playerID', 'nameFirst', 'nameLast']]

batting_w_people = pd.merge(batting_trimmed, people_trimmed, on = 'playerID', how = 'inner')
print(batting_w_people.shape)
print(batting_w_people.columns)

In [ ]:
free_agents.head(10)

In [ ]:
# Join based on nameFirst/nameLast
free_agents_batting = pd.merge(free_agents, batting_w_people, 
                               left_on = ['nameFirst', 'nameLast', 'Year'],
                               right_on = ['nameFirst', 'nameLast', 'yearID'])
print(free_agents_batting.shape)
print(free_agents_batting.columns)
print(free_agents_batting.head(10))

In [ ]:
# Pull out only the desired columns
free_agents_batting = free_agents_batting.drop(['index', 'Full_Name', 'Year'], axis = 1)
print(free_agents_batting.head(10))

## Task 2: Add positions

This will require data from our new "free_agents_batting" and "appearances". Basically:

* Pull appearances data
* Collapse "appearances" data into positions
* Join it with free_agents_batting data

In [ ]:
# Bring in Appearances data to add positions
appearances = pullFullTable('appearances', engine)
    
print(appearances.head())

In [ ]:
# Subset to only positional data and group by playerID/yearID
appearances_compact = appearances.drop(['index', 'teamID','lgID', 'G_batting', 
                                        'G_defense','G_all','GS', 'G_ph', 'G_pr'], 
                                       axis = 1).groupby(['playerID','yearID'], 
                                                         as_index = False).sum()

# Check data
print(appearances_compact.head())

In [ ]:
# Figure out primary position by melting, then grouping and finding the max value
appearances_melt = pd.melt(appearances_compact, id_vars= ['playerID', 'yearID'],
                           value_name = 'Games', var_name = 'Position')
print(appearances_melt[appearances_melt['playerID'] == 'clontbr01'])

In [ ]:
# Grab the index for the maximum games
primary_idx = appearances_melt.groupby(['playerID','yearID'])['Games'].idxmax()

# Use it to screen out the proper rows
primary_position = appearances_melt.loc[primary_idx]

# Turn the "Position" Column into the right contents by pulling just the position and capitalizing
primary_position['Position'] = primary_position.Position.str.split("_").str.get(1).str.upper()
print(primary_position[primary_position['playerID'] == 'clontbr01'])

In [ ]:
# Do the join on the 6202 x 13 free_agents_batting
# Join based on nameFirst/nameLast
fa_bat_pos = pd.merge(free_agents_batting, primary_position, 
                      on = ['playerID', 'yearID']).drop(['Games'], axis = 1)
print(fa_bat_pos.head(10))
print(fa_bat_pos.shape)

## Task 3 Add Team WAR for position

Basically I see this as:

1. Load the Team WAR data
2. Change column names to be more concise
3. Join it to the existing data frame using yearID + position. This should necessarily remove pitchers

In [ ]:
# Pull the data but drop the index
position_war = pullFullTable('position_team_war', engine).drop(['index'], axis = 1)
print(position_war.info())

In [ ]:
# Change the Year to a date-time and call it "yearID"
position_war['yearID'] = position_war.Year
position_war = position_war.drop(['Year'], axis = 1)
print(position_war.columns)

In [ ]:
# Create a dictionary for converting these to abbreviations
team_dict = {'Angels' : 'LAA', 'Astros' : 'HOU', 'Athletics' : 'OAK', 'Blue Jays' : 'TOR', 
             'Braves' : 'ATL', 'Brewers': 'MIL', 'Cardinals' : 'STL', 'Cubs' : 'CHN',
             'Diamondbacks' : 'ARI', 'Dodgers' : 'LAN', 'Giants' : 'SFN', 'Indians' : 'CLE',
             'Mariners' : 'SEA', 'Marlins' : 'MIA', 'Mets' : 'NYN', 'Nationals' : 'WAS',
             'Orioles' : 'BAL', 'Padres' : 'SDN', 'Phillies' : 'PHI', 'Pirates' : 'PIT', 
             'Rangers' : 'TEX', 'Rays' : 'TBR', 'Red Sox' : 'BOS', 'Reds' : 'CIN', 
             'Rockies' : 'COL', 'Royals' : 'KCR', 'Tigers' : 'DET', 'Twins' : 'MIN', 
             'White Sox' : 'CHA', 'Yankees' : 'NYA'}

# Alter it to include WAR
team_dict = {key : value + "_WAR" for key, value in team_dict.items()}
print(team_dict)

In [ ]:
position_war = position_war.rename(columns = team_dict)
print(position_war.head())

In [ ]:
position_war.info()

### Now add these WAR data to the batting data, by position/year


In [ ]:
fa_bat_pos.info()

In [ ]:
fa_bat_pos['Position'].value_counts()

# ALERT ALERT: Grab Pitcher info and add in to fix this!!

In [ ]:
# Merge the 2 data frames
# This is only the ~2350 position players for now; do a left join
fa_bat_team_war = pd.merge(fa_bat_pos, position_war, how = 'left',
                           on = ['Position', 'yearID'], )

print(fa_bat_team_war.info())

## Task ???: Pull Team data

Pull this to help with team -> teamID

In [ ]:
# Change column names to team abbreviations using Team Data
teams = pullFullTable('teams', engine)
print(teams.head())
print(list(teams.columns))

In [ ]:
# Pull just a handful of these columns (W, G, teamID, name, yearID)
teams_short = teams[['yearID', 'teamID', 'name', 'W', 'G']]
teams_short['teamID'].value_counts()

In [ ]:
# Convert altered names/teamID 
name_change = {'Anaheim Angels': 'Los Angeles Angels of Anaheim', 
                   'Tampa Bay Devil Rays' : 'Tampa Bay Rays',
                   'Montreal Expos' : 'Washington Nationals', 
                   'Florida Marlins' : 'Miami Marlins'
                  }
    
origin_change = {'ANA': 'LAA', 'TBD':'TBR', 'MON':'WAS', 'FLO':'MIA'}

teams_short['name'] = teams_short['name'].replace(name_change)
teams_short['teamID'] = teams_short['teamID'].replace(origin_change)

# Change W/G to W_Pct
    
teams_short['W_Pct'] = teams_short['W'].divide(teams_short.G)

teams_short = teams_short.drop(['W','G'], axis = 1)

teams_short.head()

In [ ]:
teams_short.name.value_counts()

### Next task: Use these data to change the free_agents data a bit

We need to convert the Destination data from free_agents...this is where to do it!

*This will also remove the FAs without teams*

In [ ]:
# Check destination data
fa_bat_team_war.Destination.value_counts()

In [ ]:
#Fix the weird angels data
fa_bat_team_war['Destination'] = fa_bat_team_war['Destination'].replace({'Los Angeles Angels' :
                                                                         'Los Angeles Angels of Anaheim'})
fa_bat_team_war.Destination.value_counts()

In [ ]:
# Do a join to the sub-team DF
team_translate = teams_short[['teamID', 'name']].drop_duplicates()


fa_bat_team_war_teamID = pd.merge(fa_bat_team_war, team_translate, how = 'left',
                          left_on = ['Destination'], right_on = ['name'])

# Substitute the Destination Column with the info from teamID and drop teamID
fa_bat_team_war_teamID['Destination'] = fa_bat_team_war_teamID['teamID']
fa_bat_team_war_teamID = fa_bat_team_war_teamID.drop(['teamID'], axis = 1)
fa_bat_team_war_teamID.drop_duplicates().head(10)

## Task 5: Use payroll data to cluster teams

Now I need to use payroll data to create clusters of teams. So I'll:

1. Load the payroll data
2. Standardize it for each year
3. Run it through clustering
4. Use cluster labels to create a translation

In [ ]:
# Load payroll data
payrolls = pullFullTable('payrolls', engine)
payrolls.set_index('Year', inplace=True)
payrolls.info()

In [ ]:
# Transpose the data and standardize it
payrolls_transposed = payrolls.transpose()
payrolls_transposed_standard = payrolls_transposed.transform(zscore)
payrolls_transposed_standard

In [ ]:
# Cluster them via hierarchical clustering

# Perform the necessary imports
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

# Calculate the linkage: mergings
mergings = linkage(payrolls_transposed_standard.values, method = 'complete')

# Plot the dendrogram, using varieties as labels
dendrogram(mergings,
           labels=list(payrolls_transposed_standard.index),
           leaf_rotation=90,
           leaf_font_size=6,
)
plt.show()

In [ ]:
# Select clusters using maximum height of 6
from scipy.cluster.hierarchy import fcluster

# Use fcluster to extract labels: labels
labels = fcluster(mergings, 6, criterion = 'distance')


label_data = pd.DataFrame({'label' : labels, 'Team' : list(payrolls_transposed_standard.index)})

label_data.sort_values('label')

In [ ]:
# Join this to the existing data
label_data['Team'] = label_data.replace({'Los Angeles Angels':
                                         'Los Angeles Angels of Anaheim'})

labels_as_teamID = pd.merge(label_data, team_translate, 
                            left_on = ['Team'], right_on = ['name'])[['teamID', 'label']]
fa_with_clusters = pd.merge(fa_bat_team_war_teamID, labels_as_teamID,
                            left_on = ['Destination'], right_on = ['teamID'])
fa_with_clusters.head()

In [ ]:
# Cluster them via DBscan
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.1).fit(payrolls_transposed_standard.values)

labels = db.labels_
print(labels)

# Final Task: Save the data

For now, save a test set

In [ ]:
import pickle

test_set = fa_with_clusters.dropna()
final_data = test_set

final_data.to_pickle('final_data.pickle')

# Experiments with Standardizing



## Some useful plots

Things I could plot:

* Player contract vs WAR
* Player contract vs Age